In [1]:
from dataset import Dataset
from feature_extractor import *
from monitors import *
from evaluation import Evaluator
import torch
from models import *
import pandas as pd
import os
import csv

In [2]:
batch_size = 10
device_name = 'cuda:0' if torch.cuda.is_available() else 'cpu'

/home/njemmat/anaconda3/envs/neural-network-monitoring-benchmark/lib/python3.10/site-packages/torch/cuda/__init__.py:82: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1646755853668/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
gd_id = models_gdrive_ids["resnet_cifar10"]
destination = models_path + "resnet_cifar10" + ".pth"
download_file_from_google_drive(gd_id, destination)

In [4]:
#id_dataset = "cifar10"

all_id_datasets = ["cifar10", "svhn", "cifar100"]

all_ood_datasets = [["cifar100", "svhn", "lsun"],
                    ["cifar10", "tiny_imagenet", "lsun"],
                    ["cifar10", "svhn", "lsun"]]

all_perturbations = ["brightness", "blur", "pixelization"]

all_attacks = ["fgsm", "deepfool", "pgd"]


react_clip = [0.8, 0.9, 0.95, 0.99]
monitor_temperature = [1, 50, 75, 100]


all_models = ["resnet", "densenet"]

all_layers_ids = [[32], [98]]

cv = 5

clip =0.9



In [5]:
header = ["Model", "Layer", "ID dataset", "OOD dataset", "Perturbation", "Attack",
          "Monitor", "AUPR", "AUROC", "TNR@95TPR"]

save_results_path = "Results/ICCV/"
if not os.path.exists(save_results_path):
    os.makedirs(save_results_path)
f_novelty1 = open(save_results_path + "Expe_score_test_novelty1.csv", "w", encoding="UTF8")
writer_novelty1 = csv.writer(f_novelty1, delimiter =',')
writer_novelty1.writerow(header)

save_results_path = "Results/ICCV/"
if not os.path.exists(save_results_path):
    os.makedirs(save_results_path)
f_attack1 = open(save_results_path + "Expe_score_test_attack1.csv", "w", encoding="UTF8")
writer_attack1 = csv.writer(f_attack1, delimiter =',')
writer_attack1.writerow(header)


save_results_path = "Results/ICCV/"
if not os.path.exists(save_results_path):
    os.makedirs(save_results_path)
f_pertu1 = open(save_results_path + "Expe_score_test_pertu1.csv", "w", encoding="UTF8")
writer_pertu1 = csv.writer(f_pertu1, delimiter =',')
writer_pertu1.writerow(header)

85

In [6]:
for h in range(len(all_models)):
    model = all_models[h]
    for layer in all_layers_ids[h]:
        for n in range(len(all_id_datasets)):
            id_dataset = all_id_datasets[n]
            for i in range(len(all_ood_datasets)):
                ood_dataset = all_ood_datasets[i]

                dataset_train = Dataset(id_dataset, "train", model, batch_size=batch_size)
                dataset_test = Dataset(id_dataset, "test", model, batch_size=batch_size)
                dataset_ood = Dataset(ood_dataset, "test", model, None, None, batch_size=batch_size)

                feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)
                features_train, logits_train, softmax_train, pred_train, lab_train = feature_extractor.get_features(dataset_train)
                features_test, logits_test, softmax_test, pred_test, lab_test = feature_extractor.get_features(dataset_test)
                features_ood, logits_ood, softmax_ood, pred_ood, lab_ood = feature_extractor.get_features(dataset_ood)

                eval_oms = Evaluator("oms", is_novelty=(id_dataset != ood_dataset))
                eval_oms.fit_ground_truth(lab_test, lab_ood, pred_test, pred_ood)


                # Moniteur MaxSoftmaxProbability
                monitor_max_softmax = MaxSoftmaxProbabilityMonitor()
                monitor_max_softmax.fit()

                scores_test_max_softmax = monitor_max_softmax.predict(softmax_test)
                scores_ood_max_softmax = monitor_max_softmax.predict(softmax_ood)

                aupr_max_softmax = eval_oms.get_average_precision(scores_test_max_softmax, scores_ood_max_softmax)
                auroc_max_softmax = eval_oms.get_auroc(scores_test_max_softmax, scores_ood_max_softmax)
                tnr95tpr_max_softmax = eval_oms.get_tnr_frac_tpr_oms(scores_test_max_softmax, scores_ood_max_softmax, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "MaxSoftmaxProbability", aupr_max_softmax, auroc_max_softmax, tnr95tpr_max_softmax]
                writer_novelty1.writerow(data)


                # Moniteur Maxlogits
                monitor = MaxLogitMonitor()
                monitor.fit()

                scores_test_maxlogits = monitor.predict(logits_test)
                scores_ood_maxlogits= monitor.predict(logits_ood)

                aupr_maxlogits = eval_oms.get_average_precision(scores_test_maxlogits, scores_ood_maxlogits)
                auroc_maxlogits = eval_oms.get_auroc(scores_test_maxlogits, scores_ood_maxlogits)
                tnr95tpr_maxlogits = eval_oms.get_tnr_frac_tpr_oms(scores_test_maxlogits, scores_ood_maxlogits, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "Maxlogits", aupr_maxlogits, auroc_maxlogits, tnr95tpr_maxlogits]
                writer_novelty1.writerow(data)
                
                
                # Moniteur EnergyMonitor
                monitor_energy = EnergyMonitor()
                monitor_energy.fit()

                scores_test_energy = monitor_energy.predict(logits_test)
                scores_ood_energy = monitor_energy.predict(logits_ood)

                aupr_energy = eval_oms.get_average_precision(scores_test_energy, scores_ood_energy)
                auroc_energy = eval_oms.get_auroc(scores_test_energy, scores_ood_energy)
                tnr95tpr_energy = eval_oms.get_tnr_frac_tpr_oms(scores_test_energy, scores_ood_energy, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "EnergyMonitor", aupr_energy, auroc_energy, tnr95tpr_energy]
                writer_novelty1.writerow(data)

           


                # Moniteur ODIN

                monitor_ODIN = ODINMonitor()
                monitor_ODIN.fit()

                scores_test_ODIN = monitor_ODIN.predict(logits_test)
                scores_ood_ODIN= monitor_ODIN.predict(logits_ood)

                aupr_ODIN = eval_oms.get_average_precision(scores_test_ODIN, scores_ood_ODIN)
                auroc_ODIN = eval_oms.get_auroc(scores_test_ODIN, scores_ood_ODIN)
                tnr95tpr_ODIN = eval_oms.get_tnr_frac_tpr_oms(scores_test_ODIN, scores_ood_ODIN, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "ODINMonitor", aupr_ODIN, auroc_ODIN, tnr95tpr_ODIN]
                writer_novelty1.writerow(data)

                


                # Moniteur DOCTOR mode="alpha"
                monitor_doctor_alpha = Doctor(mode="alpha")
                monitor_doctor_alpha.fit()

                scores_test_doctor_alpha = monitor_doctor_alpha.predict(softmax_test)
                scores_ood_doctor_alpha = monitor_doctor_alpha.predict(softmax_ood)

                aupr_doctor_alpha = eval_oms.get_average_precision(scores_test_doctor_alpha, scores_ood_doctor_alpha)
                auroc_doctor_alpha = eval_oms.get_auroc(scores_test_doctor_alpha, scores_ood_doctor_alpha)
                tnr95tpr_doctor_alpha = eval_oms.get_tnr_frac_tpr_oms(scores_test_doctor_alpha, scores_ood_doctor_alpha, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "DOCTOR_alpha", aupr_doctor_alpha, auroc_doctor_alpha, tnr95tpr_doctor_alpha]
                writer_novelty1.writerow(data)
                
                

                # Moniteur DOCTOR mode="beta"
                monitor_doctor = Doctor(mode="beta")
                monitor_doctor.fit()

                scores_test_doctor = monitor_doctor.predict(softmax_test)
                scores_ood_doctor = monitor_doctor.predict(softmax_ood)

                aupr_doctor = eval_oms.get_average_precision(scores_test_doctor, scores_ood_doctor)
                auroc_doctor = eval_oms.get_auroc(scores_test_doctor, scores_ood_doctor)
                tnr95tpr_doctor = eval_oms.get_tnr_frac_tpr_oms(scores_test_doctor, scores_ood_doctor, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "DOCTOR_beta", aupr_doctor, auroc_doctor, tnr95tpr_doctor]
                writer_novelty1.writerow(data)




                #moniteur react mode msp

                monitor_react = ReActMonitor(quantile_value=0.9, mode="msp")
                monitor_react.fit(feature_extractor, features_train[-1])

                scores_test_react = monitor_react.predict(features_test[-1])
                scores_ood_react = monitor_react.predict(features_ood[-1])
                
                aupr_react = eval_oms.get_average_precision(scores_test_react, scores_ood_react)
                auroc_react = eval_oms.get_auroc(scores_test_react, scores_ood_react)
                tnr95tpr_react = eval_oms.get_tnr_frac_tpr_oms(scores_test_react, scores_ood_react, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "ReAct mode(MSP) ", aupr_react, auroc_react, tnr95tpr_react]
                writer_novelty1.writerow(data)
                
                
                
                
                # Moniteur ReActMonitor mode maxlogits
                monitor_react = ReActMonitor(quantile_value=clip, mode="maxlogits")
                monitor_react.fit(feature_extractor, features_train[-1])

                scores_test_react = monitor_react.predict(features_test[-1])
                scores_ood_react = monitor_react.predict(features_ood[-1])

                aupr_react = eval_oms.get_average_precision(scores_test_react, scores_ood_react)
                auroc_react = eval_oms.get_auroc(scores_test_react, scores_ood_react)
                tnr95tpr_react = eval_oms.get_tnr_frac_tpr_oms(scores_test_react, scores_ood_react, frac=0.95)

                
                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "ReAct mode(Maxlogits) ", aupr_react, auroc_react, tnr95tpr_react]
                writer_novelty1.writerow(data)
                
                
                                

                # Moniteur ReActMonitor mode energy
                monitor = ReActMonitor(quantile_value=0.9, mode = "energy")
                monitor.fit(feature_extractor, features_train[-1])

                scores_test = monitor.predict(features_test[-1])
                scores_ood = monitor.predict(features_ood[-1])

                aupr = eval_oms.get_average_precision(scores_test, scores_ood)
                auroc = eval_oms.get_auroc(scores_test, scores_ood)
                tnr95tpr = eval_oms.get_tnr_frac_tpr_oms(scores_test, scores_ood, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "ReAct mode(Energy)", aupr_react, auroc_react, tnr95tpr_react]
                writer_novelty1.writerow(data)



                # Moniteur ReActMonitor mode "ODIN"

                monitor_react_odin = ReActMonitor(quantile_value=0.9, mode = "ODIN")
                monitor_react_odin.fit(feature_extractor, features_train[-1])

                scores_test_react_odin = monitor_react_odin.predict(features_test[-1])
                scores_ood_react_odin = monitor_react_odin.predict(features_ood[-1])

                aupr_react_odin = eval_oms.get_average_precision(scores_test_react_odin, scores_ood_react_odin)
                auroc_react_odin = eval_oms.get_auroc(scores_test_react_odin, scores_ood_react_odin)
                tnr95tpr_react_odin = eval_oms.get_tnr_frac_tpr_oms(scores_test_react_odin, scores_ood_react_odin, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "ReAct mode(ODIN)", aupr_react_odin, auroc_react_odin, tnr95tpr_react_odin]
                writer_novelty1.writerow(data)


                # Moniteur ReActMonitor mode "doctor alpha"

                monitor_react_doctor = ReActMonitor(quantile_value=0.9, mode = "doctor alpha")
                monitor_react_doctor.fit(feature_extractor, features_train[-1])

                scores_test_react_doctor = monitor_react_doctor.predict(features_test[-1])
                scores_ood_react_doctor = monitor_react_doctor.predict(features_ood[-1])

                aupr_react_doctor = eval_oms.get_average_precision(scores_test_react_doctor, scores_ood_react_doctor)
                auroc_react_doctor = eval_oms.get_auroc(scores_test_react_doctor, scores_ood_react_doctor)
                tnr95tpr_react_doctor = eval_oms.get_tnr_frac_tpr_oms(scores_test_react_doctor, scores_ood_react_doctor, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(None), str(None), "ReAct mode(DOCTOR)", aupr_react_doctor, auroc_react_doctor, tnr95tpr_react_doctor]
                writer_novelty1.writerow(data)
                    


    
            for k in range(len(all_attacks)):
                ood_dataset = id_dataset

                adversarial_attack = all_attacks[k]


                dataset_train = Dataset(id_dataset, "train", model, batch_size=batch_size)
                dataset_test = Dataset(id_dataset, "test", model, batch_size=batch_size)
                dataset_ood = Dataset(id_dataset, "test", model, None, adversarial_attack, batch_size=batch_size)

                feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)

                features_train, logits_train, softmax_train, \
                    pred_train, lab_train = feature_extractor.get_features(dataset_train)
                features_test, logits_test, softmax_test, \
                    pred_test, lab_test = feature_extractor.get_features(dataset_test)
                features_ood, logits_ood, softmax_ood, \
                    pred_ood, lab_ood = feature_extractor.get_features(dataset_ood)

                eval_oms = Evaluator("oms", is_novelty=(id_dataset != ood_dataset))
                eval_oms.fit_ground_truth(lab_test, lab_ood, pred_test, pred_ood)



                # Moniteur MSP
                monitor = MaxSoftmaxProbabilityMonitor()
                monitor.fit()

                scores_test = monitor.predict(softmax_test)
                scores_ood = monitor.predict(softmax_ood)

                aupr = eval_oms.get_average_precision(scores_test, scores_ood)
                auroc = eval_oms.get_auroc(scores_test, scores_ood)
                tnr95tpr = eval_oms.get_tnr_frac_tpr_oms(scores_test, scores_ood, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "MSP",  aupr, auroc, tnr95tpr]
                writer_attack1.writerow(data)

                
                # Moniteur Maxlogits
                monitor = MaxLogitMonitor()
                monitor.fit()

                scores_test_maxlogits = monitor.predict(logits_test)
                scores_ood_maxlogits= monitor.predict(logits_ood)

                aupr_maxlogits = eval_oms.get_average_precision(scores_test_maxlogits, scores_ood_maxlogits)
                auroc_maxlogits = eval_oms.get_auroc(scores_test_maxlogits, scores_ood_maxlogits)
                tnr95tpr_maxlogits = eval_oms.get_tnr_frac_tpr_oms(scores_test_maxlogits, scores_ood_maxlogits, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "Maxlogits", aupr_maxlogits, auroc_maxlogits, tnr95tpr_maxlogits]
                writer_attack1.writerow(data)
                
      
                

                # Moniteur EnergyMonitor
                monitor_energy = EnergyMonitor()
                monitor_energy.fit()

                scores_test_energy = monitor_energy.predict(logits_test)
                scores_ood_energy = monitor_energy.predict(logits_ood)

                aupr_energy = eval_oms.get_average_precision(scores_test_energy, scores_ood_energy)
                auroc_energy = eval_oms.get_auroc(scores_test_energy, scores_ood_energy)
                tnr95tpr_energy = eval_oms.get_tnr_frac_tpr_oms(scores_test_energy, scores_ood_energy, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "EnergyMonitor", aupr_energy, auroc_energy, tnr95tpr_energy]
                writer_attack1.writerow(data)

                # Moniteur ODIN
                monitor_ODIN = ODINMonitor()
                monitor_ODIN.fit()

                scores_test_ODIN = monitor_ODIN.predict(logits_test)
                scores_ood_ODIN= monitor_ODIN.predict(logits_ood)

                aupr_ODIN = eval_oms.get_average_precision(scores_test_ODIN, scores_ood_ODIN)
                auroc_ODIN = eval_oms.get_auroc(scores_test_ODIN, scores_ood_ODIN)
                tnr95tpr_ODIN = eval_oms.get_tnr_frac_tpr_oms(scores_test_ODIN, scores_ood_ODIN, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "ODINMonitor", aupr_ODIN, auroc_ODIN, tnr95tpr_ODIN]
                writer_attack1.writerow(data)



                # Moniteur DOCTOR mode="alpha"
                monitor_doctor_alpha = Doctor(mode="alpha")
                monitor_doctor_alpha.fit()

                scores_test_doctor_alpha = monitor_doctor_alpha.predict(softmax_test)
                scores_ood_doctor_alpha = monitor_doctor_alpha.predict(softmax_ood)

                aupr_doctor_alpha = eval_oms.get_average_precision(scores_test_doctor_alpha, scores_ood_doctor_alpha)
                auroc_doctor_alpha = eval_oms.get_auroc(scores_test_doctor_alpha, scores_ood_doctor_alpha)
                tnr95tpr_doctor_alpha = eval_oms.get_tnr_frac_tpr_oms(scores_test_doctor_alpha, scores_ood_doctor_alpha, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "DOCTOR_alpha", aupr_doctor_alpha, auroc_doctor_alpha, tnr95tpr_doctor_alpha]
                writer_attack1.writerow(data)


                # Moniteur DOCTOR mode="beta"
                monitor_doctor = Doctor(mode="beta")
                monitor_doctor.fit()

                scores_test_doctor = monitor_doctor.predict(softmax_test)
                scores_ood_doctor = monitor_doctor.predict(softmax_ood)

                aupr_doctor = eval_oms.get_average_precision(scores_test_doctor, scores_ood_doctor)
                auroc_doctor = eval_oms.get_auroc(scores_test_doctor, scores_ood_doctor)
                tnr95tpr_doctor = eval_oms.get_tnr_frac_tpr_oms(scores_test_doctor, scores_ood_doctor, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "DOCTOR_beta",  aupr_doctor, auroc_doctor, tnr95tpr_doctor]
                writer_attack1.writerow(data)


               
                # Moniteur ReActMonitor MSP
                monitor_react = ReActMonitor(quantile_value=0.9, mode="msp")
                monitor_react.fit(feature_extractor, features_train[-1])

                scores_test_react = monitor_react.predict(features_test[-1])
                scores_ood_react = monitor_react.predict(features_ood[-1])

                aupr_react = eval_oms.get_average_precision(scores_test_react, scores_ood_react)
                auroc_react = eval_oms.get_auroc(scores_test_react, scores_ood_react)
                tnr95tpr_react = eval_oms.get_tnr_frac_tpr_oms(scores_test_react, scores_ood_react, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "ReAct mode(MSP)", aupr_react, auroc_react, tnr95tpr_react]
                writer_attack1.writerow(data)
                                
                # Moniteur ReActMonitor mode maxlogits
                monitor_react = ReActMonitor(quantile_value=clip, mode="maxlogits")
                monitor_react.fit(feature_extractor, features_train[-1])

                scores_test_react = monitor_react.predict(features_test[-1])
                scores_ood_react = monitor_react.predict(features_ood[-1])

                aupr_react = eval_oms.get_average_precision(scores_test_react, scores_ood_react)
                auroc_react = eval_oms.get_auroc(scores_test_react, scores_ood_react)
                tnr95tpr_react = eval_oms.get_tnr_frac_tpr_oms(scores_test_react, scores_ood_react, frac=0.95)

                
                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "ReAct mode(Maxlogits) ", aupr_react, auroc_react, tnr95tpr_react]
                writer_attack1.writerow(data)


                # Moniteur ReActMonitor mode energy

                monitor = ReActMonitor(quantile_value=0.9, mode = "energy")
                monitor.fit(feature_extractor, features_train[-1])

                scores_test = monitor.predict(features_test[-1])
                scores_ood = monitor.predict(features_ood[-1])

                aupr = eval_oms.get_average_precision(scores_test, scores_ood)
                auroc = eval_oms.get_auroc(scores_test, scores_ood)
                tnr95tpr = eval_oms.get_tnr_frac_tpr_oms(scores_test, scores_ood, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "ReAct mode(Energy)", aupr, auroc, tnr95tpr]
                writer_attack1.writerow(data) 


                # Moniteur ReActMonitor mode "ODIN"

                monitor_react_odin = ReActMonitor(quantile_value=0.9, mode = "ODIN")
                monitor_react_odin.fit(feature_extractor, features_train[-1])

                scores_test_react_odin = monitor_react_odin.predict(features_test[-1])
                scores_ood_react_odin = monitor_react_odin.predict(features_ood[-1])

                aupr_react_odin = eval_oms.get_average_precision(scores_test_react_odin, scores_ood_react_odin)
                auroc_react_odin = eval_oms.get_auroc(scores_test_react_odin, scores_ood_react_odin)
                tnr95tpr_react_odin = eval_oms.get_tnr_frac_tpr_oms(scores_test_react_odin, scores_ood_react_odin, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "ReAct mode(ODIN)", aupr_react_odin, auroc_react_odin, tnr95tpr_react_odin]
                writer_attack1.writerow(data)


                # Moniteur ReActMonitor mode "doctor alpha"

                monitor_react_doctor = ReActMonitor(quantile_value=0.9, mode = "doctor alpha")
                monitor_react_doctor.fit(feature_extractor, features_train[-1])

                scores_test_react_doctor = monitor_react_doctor.predict(features_test[-1])
                scores_ood_react_doctor = monitor_react_doctor.predict(features_ood[-1])

                aupr_react_doctor = eval_oms.get_average_precision(scores_test_react_doctor, scores_ood_react_doctor)
                auroc_react_doctor = eval_oms.get_auroc(scores_test_react_doctor, scores_ood_react_doctor)
                tnr95tpr_react_doctor = eval_oms.get_tnr_frac_tpr_oms(scores_test_react_doctor, scores_ood_react_doctor, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(None), str(adversarial_attack), "ReAct mode(DOCTOR)", aupr_react_doctor, auroc_react_doctor, tnr95tpr_react_doctor]
                writer_attack1.writerow(data)



            for j in range(len(all_perturbations)):
                ood_dataset = id_dataset

                additional_transform = all_perturbations[j]


                dataset_train = Dataset(id_dataset, "train", model, batch_size=batch_size)
                dataset_test = Dataset(id_dataset, "test", model, batch_size=batch_size)
                dataset_ood = Dataset(id_dataset, "test", model, additional_transform, None, batch_size=batch_size)

                feature_extractor = FeatureExtractor(model, id_dataset, [layer], device_name)

                features_train, logits_train, softmax_train, \
                    pred_train, lab_train = feature_extractor.get_features(dataset_train)
                features_test, logits_test, softmax_test, \
                    pred_test, lab_test = feature_extractor.get_features(dataset_test)
                features_ood, logits_ood, softmax_ood, \
                    pred_ood, lab_ood = feature_extractor.get_features(dataset_ood)

                eval_oms = Evaluator("oms", is_novelty=(id_dataset != ood_dataset))
                eval_oms.fit_ground_truth(lab_test, lab_ood, pred_test, pred_ood)


                # Moniteur MSP
                monitor = MaxSoftmaxProbabilityMonitor()
                monitor.fit()

                scores_test = monitor.predict(softmax_test)
                scores_ood = monitor.predict(softmax_ood)

                aupr = eval_oms.get_average_precision(scores_test, scores_ood)
                auroc = eval_oms.get_auroc(scores_test, scores_ood)
                tnr95tpr = eval_oms.get_tnr_frac_tpr_oms(scores_test, scores_ood, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "MSP", aupr, auroc, tnr95tpr]
                writer_pertu1.writerow(data)
                

                # Moniteur Maxlogits
                monitor = MaxLogitMonitor()
                monitor.fit()

                scores_test_maxlogits = monitor.predict(logits_test)
                scores_ood_maxlogits= monitor.predict(logits_ood)

                aupr_maxlogits = eval_oms.get_average_precision(scores_test_maxlogits, scores_ood_maxlogits)
                auroc_maxlogits = eval_oms.get_auroc(scores_test_maxlogits, scores_ood_maxlogits)
                tnr95tpr_maxlogits = eval_oms.get_tnr_frac_tpr_oms(scores_test_maxlogits, scores_ood_maxlogits, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "Maxlogits", aupr_maxlogits, auroc_maxlogits, tnr95tpr_maxlogits]
                writer_pertu1.writerow(data)
                


               
                # Moniteur EnergyMonitor
                monitor_energy = EnergyMonitor(temperature=1)
                monitor_energy.fit()

                scores_test_energy = monitor_energy.predict(logits_test)
                scores_ood_energy = monitor_energy.predict(logits_ood)

                aupr_energy = eval_oms.get_average_precision(scores_test_energy, scores_ood_energy)
                auroc_energy = eval_oms.get_auroc(scores_test_energy, scores_ood_energy)
                tnr95tpr_energy = eval_oms.get_tnr_frac_tpr_oms(scores_test_energy, scores_ood_energy, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "EnergyMonitor", aupr_energy, auroc_energy, tnr95tpr_energy]
                writer_pertu1.writerow(data)

                # Moniteur ODIN
                monitor_ODIN = ODINMonitor(temperature=100)
                monitor_ODIN.fit()

                scores_test_ODIN = monitor_ODIN.predict(logits_test)
                scores_ood_ODIN= monitor_ODIN.predict(logits_ood)

                aupr_ODIN = eval_oms.get_average_precision(scores_test_ODIN, scores_ood_ODIN)
                auroc_ODIN = eval_oms.get_auroc(scores_test_ODIN, scores_ood_ODIN)
                tnr95tpr_ODIN = eval_oms.get_tnr_frac_tpr_oms(scores_test_ODIN, scores_ood_ODIN, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "ODINMonitor", aupr_ODIN, auroc_ODIN, tnr95tpr_ODIN]
                writer_pertu1.writerow(data)


                # Moniteur DOCTOR mode="alpha"
                monitor_doctor_alpha = Doctor(mode="alpha")
                monitor_doctor_alpha.fit()

                scores_test_doctor_alpha = monitor_doctor_alpha.predict(softmax_test)
                scores_ood_doctor_alpha = monitor_doctor_alpha.predict(softmax_ood)

                aupr_doctor_alpha = eval_oms.get_average_precision(scores_test_doctor_alpha, scores_ood_doctor_alpha)
                auroc_doctor_alpha = eval_oms.get_auroc(scores_test_doctor_alpha, scores_ood_doctor_alpha)
                tnr95tpr_doctor_alpha = eval_oms.get_tnr_frac_tpr_oms(scores_test_doctor_alpha, scores_ood_doctor_alpha, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "DOCTOR_alpha", aupr_doctor_alpha, auroc_doctor_alpha, tnr95tpr_doctor_alpha]
                writer_pertu1.writerow(data)


                # Moniteur DOCTOR mode="beta"
                monitor_doctor = Doctor(mode="beta")
                monitor_doctor.fit()

                scores_test_doctor = monitor_doctor.predict(softmax_test)
                scores_ood_doctor = monitor_doctor.predict(softmax_ood)

                aupr_doctor = eval_oms.get_average_precision(scores_test_doctor, scores_ood_doctor)
                auroc_doctor = eval_oms.get_auroc(scores_test_doctor, scores_ood_doctor)
                tnr95tpr_doctor = eval_oms.get_tnr_frac_tpr_oms(scores_test_doctor, scores_ood_doctor, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "DOCTOR_beta", aupr_doctor, auroc_doctor, tnr95tpr_doctor]
                writer_pertu1.writerow(data)


                

                # Moniteur ReActMonitor MSP
                monitor_react = ReActMonitor(quantile_value=0.9, mode="msp")
                monitor_react.fit(feature_extractor, features_train[-1])

                scores_test_react = monitor_react.predict(features_test[-1])
                scores_ood_react = monitor_react.predict(features_ood[-1])

                aupr_react = eval_oms.get_average_precision(scores_test_react, scores_ood_react)
                auroc_react = eval_oms.get_auroc(scores_test_react, scores_ood_react)
                tnr95tpr_react = eval_oms.get_tnr_frac_tpr_oms(scores_test_react, scores_ood_react, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "ReAct mode(MSP)", aupr_react, auroc_react, tnr95tpr_react]
                writer_pertu1.writerow(data)
                                
                # Moniteur ReActMonitor mode maxlogits
                monitor_react = ReActMonitor(quantile_value=clip, mode="maxlogits")
                monitor_react.fit(feature_extractor, features_train[-1])

                scores_test_react = monitor_react.predict(features_test[-1])
                scores_ood_react = monitor_react.predict(features_ood[-1])

                aupr_react = eval_oms.get_average_precision(scores_test_react, scores_ood_react)
                auroc_react = eval_oms.get_auroc(scores_test_react, scores_ood_react)
                tnr95tpr_react = eval_oms.get_tnr_frac_tpr_oms(scores_test_react, scores_ood_react, frac=0.95)

                
                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "ReAct mode(Maxlogits) ", aupr_react, auroc_react, tnr95tpr_react]
                writer_pertu1.writerow(data)


                # Moniteur ReActMonitor mode energy
                monitor = ReActMonitor(quantile_value=0.9, mode = "energy")
                monitor.fit(feature_extractor, features_train[-1])

                scores_test = monitor.predict(features_test[-1])
                scores_ood = monitor.predict(features_ood[-1])

                aupr = eval_oms.get_average_precision(scores_test, scores_ood)
                auroc = eval_oms.get_auroc(scores_test, scores_ood)
                tnr95tpr = eval_oms.get_tnr_frac_tpr_oms(scores_test, scores_ood, frac=0.95)

                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "ReAct mode(Energy)", aupr, auroc, tnr95tpr]
                writer_pertu1.writerow(data)


                # Moniteur ReActMonitor mode "ODIN"

                monitor_react_odin = ReActMonitor(quantile_value=0.9, mode = "ODIN")
                monitor_react_odin.fit(feature_extractor, features_train[-1])

                scores_test_react_odin = monitor_react_odin.predict(features_test[-1])
                scores_ood_react_odin = monitor_react_odin.predict(features_ood[-1])

                aupr_react_odin = eval_oms.get_average_precision(scores_test_react_odin, scores_ood_react_odin)
                auroc_react_odin = eval_oms.get_auroc(scores_test_react_odin, scores_ood_react_odin)
                tnr95tpr_react_odin = eval_oms.get_tnr_frac_tpr_oms(scores_test_react_odin, scores_ood_react_odin, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "ReAct mode(ODIN)", aupr_react_odin, auroc_react_odin, tnr95tpr_react_odin]
                writer_pertu1.writerow(data)


                # Moniteur ReActMonitor mode "doctor alpha"

                monitor_react_doctor = ReActMonitor(quantile_value=0.9, mode = "doctor alpha")
                monitor_react_doctor.fit(feature_extractor, features_train[-1])

                scores_test_react_doctor = monitor_react_doctor.predict(features_test[-1])
                scores_ood_react_doctor = monitor_react_doctor.predict(features_ood[-1])

                aupr_react_doctor = eval_oms.get_average_precision(scores_test_react_doctor, scores_ood_react_doctor)
                auroc_react_doctor = eval_oms.get_auroc(scores_test_react_doctor, scores_ood_react_doctor)
                tnr95tpr_react_doctor = eval_oms.get_tnr_frac_tpr_oms(scores_test_react_doctor, scores_ood_react_doctor, frac=0.95)


                data = [model, layer, id_dataset, ood_dataset, str(additional_transform), str(None), "ReAct mode(DOCTOR)", aupr_react_doctor, auroc_react_doctor, tnr95tpr_react_doctor]
                writer_pertu1.writerow(data)

f_novelty1.close()
f_attack1.close()
f_pertu1.close()


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Using downloaded and verified file: ./Data/test_32x32.mat
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and v

In [7]:
df_novelty1 = pd.read_csv(save_results_path + "Expe_score_test_novelty1.csv")

In [8]:
df_novelty1.head(100)

,Model,Layer,ID dataset,OOD dataset,Perturbation,Attack,Monitor,AUPR,AUROC,TNR@95TPR
0,resnet,32,cifar10,cifar100,None,None,MaxSoftmaxProbability,0.897585,0.896235,0.553326
1,resnet,32,cifar10,cifar100,None,None,Maxlogits,0.907369,0.897087,0.507526
2,resnet,32,cifar10,cifar100,None,None,EnergyMonitor,0.908303,0.897568,0.507526
3,resnet,32,cifar10,cifar100,None,None,ODINMonitor,0.907386,0.897219,0.508701
4,resnet,32,cifar10,cifar100,None,None,DOCTOR_alpha,0.899613,0.896619,0.553326
...,...,...,...,...,...,...,...,...,...,...
95,resnet,32,cifar100,lsun,None,None,ReAct mode(Maxlogits),0.876542,0.832343,0.337376
96,resnet,32,cifar100,lsun,None,None,ReAct mode(Energy),0.876542,0.832343,0.337376
97,resnet,32,cifar100,lsun,None,None,ReAct mode(ODIN),0.876579,0.832445,0.338014
98,resnet,32,cifar100,lsun,None,None,ReAct mode(DOCTOR),0.880630,0.837671,0.346311


In [9]:
df_attack1 = pd.read_csv(save_results_path + "Expe_score_test_attack1.csv")
df_attack1.head(50)

,Model,Layer,ID dataset,OOD dataset,Perturbation,Attack,Monitor,AUPR,AUROC,TNR@95TPR
0,resnet,32,cifar10,cifar10,None,fgsm,MSP,0.641993,0.871017,0.604469
1,resnet,32,cifar10,cifar10,None,fgsm,Maxlogits,0.663020,0.875133,0.571084
2,resnet,32,cifar10,cifar10,None,fgsm,EnergyMonitor,0.664001,0.875325,0.571151
3,resnet,32,cifar10,cifar10,None,fgsm,ODINMonitor,0.662922,0.875182,0.572289
4,resnet,32,cifar10,cifar10,None,fgsm,DOCTOR_alpha,0.647358,0.871571,0.604536
5,resnet,32,cifar10,cifar10,None,fgsm,DOCTOR_beta,0.641993,0.871017,0.604469
6,resnet,32,cifar10,cifar10,None,fgsm,ReAct mode(MSP),0.625406,0.853244,0.467585
7,resnet,32,cifar10,cifar10,None,fgsm,ReAct mode(Maxlogits),0.634322,0.848009,0.405700
8,resnet,32,cifar10,cifar10,None,fgsm,ReAct mode(Energy),0.636193,0.848007,0.405433
9,resnet,32,cifar10,cifar10,None,fgsm,ReAct mode(ODIN),0.634381,0.848208,0.409179


In [10]:
df_pertu1 = pd.read_csv(save_results_path + "Expe_score_test_pertu1.csv")
df_pertu1.head(50)

,Model,Layer,ID dataset,OOD dataset,Perturbation,Attack,Monitor,AUPR,AUROC,TNR@95TPR
0,resnet,32,cifar10,cifar10,brightness,None,MSP,0.645866,0.869905,0.572330
1,resnet,32,cifar10,cifar10,brightness,None,Maxlogits,0.670675,0.875879,0.552317
2,resnet,32,cifar10,cifar10,brightness,None,EnergyMonitor,0.672417,0.876266,0.551981
3,resnet,32,cifar10,cifar10,brightness,None,ODINMonitor,0.670572,0.875883,0.552317
4,resnet,32,cifar10,cifar10,brightness,None,DOCTOR_alpha,0.651583,0.870643,0.572465
5,resnet,32,cifar10,cifar10,brightness,None,DOCTOR_beta,0.645866,0.869905,0.572330
6,resnet,32,cifar10,cifar10,brightness,None,ReAct mode(MSP),0.624830,0.845123,0.359906
7,resnet,32,cifar10,cifar10,brightness,None,ReAct mode(Maxlogits),0.636483,0.839785,0.330154
8,resnet,32,cifar10,cifar10,brightness,None,ReAct mode(Energy),0.639959,0.839994,0.330020
9,resnet,32,cifar10,cifar10,brightness,None,ReAct mode(ODIN),0.636433,0.839990,0.330557
